In previous work, I had issues in mergin docking and strain data due to duplication issues. 

It is easier to deduplicate in a seperate session and have the final files for use in the Enrichment_Metrics.ipynb. 

The duplicates were caused by LigPrep, beginning at the `actives.smi` and `inactives.smi` files. 

The `actives.smi` file has defined stereochemistry (likely where known) and protomers (assuming some sort of physiological pH) protonation. `inactives.smi` does as well, which implies to me there was some sort of stereochemical and physiological pH protonation performed and saved as the underlying smiles. 

In [7]:
!echo "Protonation and stereochemistry found by linecount in actives.smi"
!grep '\+' actives.smi | wc -l
!grep '@' actives.smi | wc -l

!echo "Protonation and stereochemistry found by linecount in inactives.smi"
!grep '\+' inactives.smi | wc -l
!grep '@' inactives.smi | wc -l


Protation and stereochemistry found by linecount in actives.smi
       8
       3
Protation and stereochemistry found by linecount in inactives.smi
   64226
    6412


However, there are cases where stereochemistry is unassigned. 

In [18]:
def check_unassigned_stereocenters(filename):
    """
    Reads an .smi file and checks for unassigned stereocenters in the first 100 molecules.

    Parameters:
    filename (str): The name of the .smi file.

    Returns:
    None
    """
    supplier = Chem.SmilesMolSupplier(filename, delimiter=' ', titleLine=False)

    for i, mol in enumerate(supplier):
        if i >= 100:
            break
        if mol is not None:
            chiral_centers = Chem.FindMolChiralCenters(mol, includeUnassigned=True)
            if len(chiral_centers) > 0:
                print(f"Molecule {mol.GetProp('_Name')} has unassigned stereocenters.")

check_unassigned_stereocenters('actives.smi')

Molecule 4242585 has unassigned stereocenters.
Molecule 4248922 has unassigned stereocenters.
Molecule 11532953 has unassigned stereocenters.
Molecule 14721919 has unassigned stereocenters.
Molecule 14733740 has unassigned stereocenters.
Molecule 24781607 has unassigned stereocenters.
Molecule 24781610 has unassigned stereocenters.
Molecule 24824326 has unassigned stereocenters.
Molecule 26732625 has unassigned stereocenters.
Molecule 49819640 has unassigned stereocenters.
Molecule 51085197 has unassigned stereocenters.


In [19]:
check_unassigned_stereocenters('inactives.smi')

Molecule 51090810 has unassigned stereocenters.
Molecule 51090809 has unassigned stereocenters.
Molecule 51090784 has unassigned stereocenters.
Molecule 51090782 has unassigned stereocenters.
Molecule 51090781 has unassigned stereocenters.
Molecule 51090779 has unassigned stereocenters.
Molecule 51090748 has unassigned stereocenters.
Molecule 51090729 has unassigned stereocenters.
Molecule 51090728 has unassigned stereocenters.
Molecule 51090722 has unassigned stereocenters.
Molecule 51090699 has unassigned stereocenters.
Molecule 51090693 has unassigned stereocenters.
Molecule 51090692 has unassigned stereocenters.
Molecule 51090689 has unassigned stereocenters.
Molecule 51090672 has unassigned stereocenters.
Molecule 51090666 has unassigned stereocenters.
Molecule 51090659 has unassigned stereocenters.
Molecule 51090658 has unassigned stereocenters.
Molecule 51090654 has unassigned stereocenters.
Molecule 51090630 has unassigned stereocenters.
Molecule 51090628 has unassigned stereoc

So, in the future, we should revise our LigPrep to only enumerate unassigned stereochemistry, but keep the given protomer. I can discuss this more with the team. It is unlikely that all the stereoisomers of a given chiral active are 'true positives' (which will affect enrichment), but this is not easily assessed without checking the LIT-PCBA methodology or checking the source of the chiral active itself (was it resolved and tested?). For now, we will accept this inaccuracy until the team can have their input. 

For now, to do our Enrichment metrics, we need to deduplicate the data. The biggest issue is when we pd.merge() the docking and strain data that both contain duplicate Molecule_Names, which leads to an explosion to duplicates. So, we need to drop the duplicates from both sets prior to the merge. 

The SDF files needs to be deduplicated by Molecule_Name. We could try to do something like a straightforward 'deduplicate by keeping the best docking score', but that could have weird behavior (like active chirality issue above). Still, this is a fast method to get to Enrichment Metrics. A more complicated approach is to find some retained data/metric that has the protomer probability. 

Let's start by dropping by docking score. 

In [21]:
import pickle 

def load_from_pickle(filename):
    with open(filename, "rb") as f:
        df = pickle.load(f)
    return df

# Load from pickle
active = load_from_pickle("67B3_actives.pkl")
inactive = load_from_pickle("67B3_inactives.pkl")

In [22]:
# Convert the 'Title' column to a string in both dataframes
active["Title"] = active["Title"].astype(str)
inactive["Title"] = inactive["Title"].astype(str)

# Rename the 'Title' column to 'Molecule_Name' in both dataframes
active.rename(columns={"Title": "Molecule_Name"}, inplace=True)
inactive.rename(columns={"Title": "Molecule_Name"}, inplace=True)

In [28]:
#Sort to retain by lowest docking score, lowest to highest (asecending)
actives_docking = active.sort_values('r_i_docking_score').copy()
inactives_docking = inactive.sort_values('r_i_docking_score').copy()

display(actives_docking['r_i_docking_score'].head())
display(inactives_docking['r_i_docking_score'].head())

0   -8.36082
1   -8.31699
2   -7.92756
3   -7.82703
4   -7.62963
Name: r_i_docking_score, dtype: float64

0   -11.3790
1   -11.0561
2   -10.9274
3   -10.8396
4   -10.6551
Name: r_i_docking_score, dtype: float64

In [29]:
# Drop duplicates based on the 'Molecule_Name' column in both dataframes
# Keep the first instance of the duplicate (lowest docking score)
actives_docking.drop_duplicates(subset='Molecule_Name', keep='first', inplace=True)
inactives_docking.drop_duplicates(subset='Molecule_Name', keep='first', inplace=True)


Now, we need to write them back as SDF files, so we can use the torsion scripts. This is definitely not ideal, but as a first pass this is fine. We'll use the Mol column we have. 

In [32]:
actives_docking.head(n=1)

,Molecule_Name,Mol,Activity,i_epik_Tot_Q,i_epik_Tot_abs_Q,i_i_glide_confnum,i_i_glide_lignum,i_i_glide_posenum,i_i_glide_rotatable_bonds,i_lp_mmshare_version,...,r_i_glide_rewards,r_lp_Energy,r_lp_tautomer_probability,s_epik_cmdline,s_epik_input,s_i_glide_gridfile,s_lp_Force_Field,s_lp_Variant,s_m_source_file,s_epik_Chemistry_Notes
0,49819640,<rdkit.Chem.rdchem.Mol object at 0x28ed24130>,1,1,1,152,33,258,9,53161,...,-1.62181,38.239346,1.0,J2VwaWtfcHl0aG9uJywgJy1waCcsICc3LjAnLCAnLXRuJy...,W2NIXTFbY0hdYyhGKVtjSF1bY0hdYzFbTkhdQyg9TylbQ0...,glide-grid_67B3_PL_Complex,S-OPLS,49819640-1,actives.smi,NaN


In [33]:
active.columns

Index(['Molecule_Name', 'Mol', 'Activity', 'i_epik_Tot_Q', 'i_epik_Tot_abs_Q',
       'i_i_glide_confnum', 'i_i_glide_lignum', 'i_i_glide_posenum',
       'i_i_glide_rotatable_bonds', 'i_lp_mmshare_version',
       'i_m_source_file_index', 'i_f3d_flags', 'b_lp_Chiralities_Consistent',
       'r_epik_Charging_Adjusted_Penalty', 'r_epik_Ionization_Penalty',
       'r_epik_Ionization_Penalty_Charging',
       'r_epik_Ionization_Penalty_Neutral', 'r_epik_State_Penalty',
       'r_i_docking_score', 'r_i_glide_ecoul', 'r_i_glide_eff_state_penalty',
       'r_i_glide_einternal', 'r_i_glide_emodel', 'r_i_glide_energy',
       'r_i_glide_erotb', 'r_i_glide_esite', 'r_i_glide_evdw',
       'r_i_glide_gscore', 'r_i_glide_hbond', 'r_i_glide_ligand_efficiency',
       'r_i_glide_ligand_efficiency_ln', 'r_i_glide_ligand_efficiency_sa',
       'r_i_glide_lipo', 'r_i_glide_metal', 'r_i_glide_rewards', 'r_lp_Energy',
       'r_lp_tautomer_probability', 's_epik_cmdline', 's_epik_input',
       's_i_glid

The issue is writing the sdf is going to take forever and I don't want to do that. Then, I'll need to load those sdfs back as dataframes and concat them, which is also going to be really annoying. 

I can't figure out how to drop duplicate titles directly via Schrodinger tools and their documentation is useless. So the only approach I actually have is above. Otherwise I have to do the above. 

I can just arbitarily drop them from the sdf and merge, but the strain data has duplicate entries and I have no idea how to deduplicate that in the same order it arrived. 

Inactive Docking dataset: 262197
OPRK1_inactives_strain dataset: 262190

Going through all of the effort to deduplicate by writing the sdfs -> running torsion strain -> loading the sdfs back (+ making mol objects) -> recombining it into a dataset seems like a waste of time. I think it is better to try to do the fast merge by keeping the first appearence of the title and move on.

Acutally, it's going to be very annoying regardless because everytime a stereocenter was enumerated, it did not update the title field to reflect that. I can't, at this point, know the original chirality. I can still do this merge but the results are even less reasonable. 

It really sucks but the only real way to do this now is to just enumerate the stereochemistry like I do in DeepDocking, then generate the single most probably structure by ligprep from scratch, then dock from scratch. I hate that. 

Let's do the fast merge and set up all those jobs so I can work on enrichment. 



In [34]:
def concatenate_csv_files(file_list):
    """
    Concatenates multiple strain CSV files into a single dataframe.
    Only the first five columns are kept for now. 

    Args:
        file_list (list): A list of file paths to the CSV files.

    Returns:
        pandas.DataFrame: The concatenated dataframe.
    """

    # Specify the column names
    column_names = [
        "Molecule_Name",
        "Total_E",
        "Lower_Bound",
        "Upper_Bound",
        "Num_Torsion_Patterns",
    ]

    # List to hold dataframes
    df_list = []

    # Loop over each file in the list
    for file in file_list:
        # Import the CSV file as a df, using only the first five columns of the CSV file
        df = pd.read_csv(file, usecols=range(5), names=column_names, header=0)
        df_list.append(df)

    # Concatenate all dataframes in the list
    final_df = pd.concat(df_list, ignore_index=True)

    return final_df



In [38]:
import glob
import pandas as pd

# Use glob to get csv files ('subset_*.csv) from the directory
csv_files = glob.glob("subset_*.csv")

# Sort the list of files by the number in their names
csv_files.sort(key=lambda x: int(x.split("_")[1].split(".")[0]))

# Check the order
print(csv_files)

# Concatenate the CSV files
OPRK1_inactives_strain = concatenate_csv_files(csv_files)

# We can still use concatenate_csv_files() to read in the active strain data
OPRK1_actives_strain = concatenate_csv_files(["67B3_actives.csv"])

['subset_1.csv', 'subset_2.csv', 'subset_3.csv', 'subset_4.csv', 'subset_5.csv', 'subset_6.csv', 'subset_7.csv', 'subset_8.csv', 'subset_9.csv', 'subset_10.csv']


In [39]:
# Molecule_Name is being read as a float, so convert it to a string 
OPRK1_inactives_strain["Molecule_Name"] = OPRK1_inactives_strain[
"Molecule_Name"].astype(str)
OPRK1_actives_strain["Molecule_Name"] = OPRK1_actives_strain["Molecule_Name"].astype(str)


In [40]:
# Deduplicate the dataframes by 'Molecule_Name', keep first instance
OPRK1_inactives_strain.drop_duplicates(subset='Molecule_Name', keep='first', inplace=True)
OPRK1_actives_strain.drop_duplicates(subset='Molecule_Name', keep='first', inplace=True)

In [42]:
# Write a function to check the amount of Duplicates in the 'Molecule_Name' column in a dataframe
def check_duplicates(df):
    """
    Checks the amount of duplicates in the 'Molecule_Name' column in a dataframe.

    Parameters:
    df (pandas.DataFrame): The dataframe to check.

    Returns:
    None
    """
    # Get the number of duplicates
    num_duplicates = df.duplicated(subset="Molecule_Name").sum()

    # Get the name of the input dataframe


    # Print the number of duplicates
    print(f"There are {num_duplicates} duplicates in the dataframe.")

check_duplicates(OPRK1_inactives_strain)
check_duplicates(OPRK1_actives_strain)
check_duplicates(actives_docking)
check_duplicates(inactives_docking)


There are 0 duplicates in the dataframe.
There are 0 duplicates in the dataframe.
There are 0 duplicates in the dataframe.
There are 0 duplicates in the dataframe.


In [43]:
# Merge the strain dataframes with the docking dataframes
inactives_data = pd.merge(OPRK1_inactives_strain, inactives_docking, on='Molecule_Name')
actives_data = pd.merge(OPRK1_actives_strain, actives_docking, on='Molecule_Name')


In [44]:
check_duplicates(inactives_data)
check_duplicates(actives_data)

There are 0 duplicates in the dataframe.
There are 0 duplicates in the dataframe.


In [45]:
# Concat the datafranes 
final_data = pd.concat([actives_data, inactives_data], ignore_index=True)

# Check for duplicates
check_duplicates(final_data)

There are 0 duplicates in the dataframe.


In [46]:
all_data_subset = final_data[
    [
        "Molecule_Name",
        "Mol",
        "Activity",
        "r_i_docking_score",
        "Total_E",
        "Lower_Bound",
        "Upper_Bound",
    ]
].copy(deep=True)

all_data_subset

,Molecule_Name,Mol,Activity,r_i_docking_score,Total_E,Lower_Bound,Upper_Bound
0,14742361,<rdkit.Chem.rdchem.Mol object at 0x28ed05350>,1,-8.316990,7.743401,6.452040,NaN
1,14721919,<rdkit.Chem.rdchem.Mol object at 0x28ed06ed0>,1,-7.927560,9.862600,7.793637,NaN
2,14733740,<rdkit.Chem.rdchem.Mol object at 0x12975ba10>,1,-7.827030,9.336486,7.212523,NaN
3,51085197,<rdkit.Chem.rdchem.Mol object at 0x2a55613a0>,1,-7.629630,7.962479,6.610545,inf
4,11532953,<rdkit.Chem.rdchem.Mol object at 0x2a5561440>,1,-7.589840,7.557015,5.150796,NaN
...,...,...,...,...,...,...,...
269329,26665106,<rdkit.Chem.rdchem.Mol object at 0x434dac2c0>,0,0.588781,0.988140,0.727169,1.279182
269330,860318,<rdkit.Chem.rdchem.Mol object at 0x434dcc360>,0,0.776713,4.875106,4.118975,NaN
269331,4244719,<rdkit.Chem.rdchem.Mol object at 0x434d9c3b0>,0,0.866768,6.162786,5.536448,inf
269332,26663220,<rdkit.Chem.rdchem.Mol object at 0x434dac450>,0,0.898282,4.117408,3.237282,NaN


In [47]:
check_duplicates(all_data_subset)

There are 0 duplicates in the dataframe.


In [48]:
# Write the dataframe to a pickle file 
all_data_subset.to_pickle("all_data_subset_dedup.pkl")